# Fetch the data into a website(1Mg Homeopathic) by using Selenium and BeautifulSoup
  


## Table - 2 : medicine_details
`name` - Name of the medicine

`brand_name`  - Brand name

`key_benefits` - Key Benefits area (Hair, eye, joint, skin)

`key_ingredients` - Ingredient of medicine

`rating` - user rating of the medicine

`number_of_rating`  - Number of people rated that product

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv


# Function to fetch additional information from a given URL
def fetch_additional_info(url):
    driver = webdriver.Chrome()
    driver.get(url)
    
    # Wait for some time to ensure the page is fully loaded
    driver.implicitly_wait(2)
    
    # Extract information using BeautifulSoup 
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Remove dropdown
    close_popup_button = driver.find_element(By.CLASS_NAME, 'UpdateCityModal__cancel-btn___2jWwS')
    close_popup_button.click()
    time.sleep(2)

    
    # Extracting the Name, Brand Name, Rating, Number of Rating, Key Benefits,  Key Ingredients
    name_of_medicine_element = soup.find('h1', {'class': 'ProductTitle__product-title___3QMYH'})
    name_of_medicine = name_of_medicine_element.text.strip() if name_of_medicine_element else 'NA'
    
    brand_name_element = soup.find('div', {'class': 'ProductTitle__marketer___7Wsj9'})
    brand_name = brand_name_element.text.strip() if brand_name_element else 'NA'
        
    rating_element = soup.find('div', {'class': 'RatingDisplay__ratings-container___3oUuo'})
    rating = rating_element.text.strip() if rating_element else 'NA'
    
    num_rating_element = soup.find('span', {'class': 'RatingDisplay__ratings-header___ZNj5b'})
    num_rating = num_rating_element.text.strip() if num_rating_element else 'NA'
    
  
    key_ingredients_element = soup.find('div', {'class': 'KeyIng__container___1cHvi'})
    key_ingredients = key_ingredients_element.text.strip() if key_ingredients_element else 'NA'
    
    
    key_benefits_element = soup.find('div', {'class': 'ProductHighlights__product-highlights___2jAF5'})
    key_benefits_items = key_benefits_element.find_all('li') if key_benefits_element else []
    key_benefits = [item.text.strip() for item in key_benefits_items] if key_benefits_items else 'NA'
    
    # Close the WebDriver
    driver.quit()
    
    return name_of_medicine, brand_name, rating, num_rating, key_ingredients, key_benefits




# Specify the path to your CSV file
csv_file_path = 'new_medicines_data_28.csv'


# Create a list to store all the 1Mg URLs
all_1mg_urls = []


# Read data from the CSV file and extract '1Mg_URL'
with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        url = row['1Mg_URL']
        all_1mg_urls.append(url)

        
# Create a list to store the fetched information
all_additional_info = []

# Iterate through each URL and fetch additional information
for url in all_1mg_urls:
    name, brand, rating, num_rating, key_ingredients, key_benefits = fetch_additional_info(url)
    all_additional_info.append({
        'Name': name , 
        'Brand_Name': brand, 
        'Rating' : rating, 
        'Number_of_rating' : num_rating,
        'Key Ingredients' : key_ingredients,
        'Key Benefits' :  key_benefits   
    })
# Print the fetched information
for info in all_additional_info:
    print(info)


{'Name': 'Dr Willmar Schwabe India Berberis Aquifolium Mother Tincture Q', 'Brand_Name': 'Dr Willmar Schwabe India Pvt Ltd', 'Rating': '4.4', 'Number_of_rating': '915 Ratings', 'Key Ingredients': 'NA', 'Key Benefits': ['Helps to relieve itchy scalp and severe headaches', 'Controls the eruption of pimples', 'Soothes itchy and burning sensation in the eyes', 'Supports skin health and improves its appearance']}
{'Name': 'Medisynth Kofeez SF Cough Syrup Sugar Free', 'Brand_Name': 'Medisynth Chemicals Pvt Ltd', 'Rating': '4.6', 'Number_of_rating': '10 Ratings', 'Key Ingredients': 'NA', 'Key Benefits': ['Helps to manage cough and cold', 'Prevents throat soreness', 'Sugar-free']}
{'Name': 'Medisynth Kofeez Cough Syrup', 'Brand_Name': 'Medisynth Chemicals Pvt Ltd', 'Rating': '4.5', 'Number_of_rating': '150 Ratings', 'Key Ingredients': 'NA', 'Key Benefits': ['Helps to relieve cough', 'Prevents throat soreness', 'Relieves inflammation']}
{'Name': 'Dr Willmar Schwabe Germany Cineraria Maritima Wi